## California School Testing

The data is California Assessment of Student Performance and Progress (CAASPP) Smarter Balanced Assessments released by the California Department of Education (link: https://caaspp.cde.ca.gov/sb2018/ResearchFileList)

In [2]:
import requests

## Retrieve Smarter Balanced Assessments Statewide results and save in the data directory
input_filenames = ['sb_ca2018_1_csv_v3.zip',
                   'sb_ca2017_1_csv_v2.zip',
                   'sb_ca2016_1_csv_v3.zip',
                   'sb_ca2015_1_csv_v3.zip']

for fn in input_filenames:
    url = 'http://www3.cde.ca.gov/caasppresearchfiles/2018/sb/' + fn
    year = fn[5:9]
    outfilename = 'data/{}.zip'.format(fn[:9])
    print('url, outfilename: ', url, outfilename)
    r = requests.get(url, allow_redirects=True)
    open(outfilename, 'wb').write(r.content)
    
## Retrieve the latest entity file, because we want to analyze the data to make decisions about current schools
url = 'http://www3.cde.ca.gov/caasppresearchfiles/2018/sb/sb_ca2018entities_csv.zip'
outfilename = 'data/ca_entities.zip'
r = requests.get(url, allow_redirects=True)
open(outfilename, 'wb').write(r.content)

url, outfilename:  http://www3.cde.ca.gov/caasppresearchfiles/2018/sb/sb_ca2018_1_csv_v3.zip data/sb_ca2018.zip
url, outfilename:  http://www3.cde.ca.gov/caasppresearchfiles/2018/sb/sb_ca2017_1_csv_v2.zip data/sb_ca2017.zip
url, outfilename:  http://www3.cde.ca.gov/caasppresearchfiles/2018/sb/sb_ca2016_1_csv_v3.zip data/sb_ca2016.zip
url, outfilename:  http://www3.cde.ca.gov/caasppresearchfiles/2018/sb/sb_ca2015_1_csv_v3.zip data/sb_ca2015.zip


188963

In [59]:
import os
import re
import pandas as pd
from zipfile import ZipFile

input_dir = 'data'
zip_filenames = sorted([input_dir + '/' + filename for filename in os.listdir(input_dir) if re.match('sb_ca\d+.zip', filename)], reverse=True)
test_result_dfs = []
for zip_fn in zip_filenames:
    print(zip_fn)
    zip_file = ZipFile(zip_fn)
    
    for text_file in zip_file.infolist():
        if 'entities' not in text_file.filename:
            df = pd.read_csv(zip_file.open(text_file.filename))
            if ('2016' in zip_fn or '2015' in zip_fn):
                df.drop(['Total CAASPP Enrollment'], axis=1, inplace=True)
            test_result_dfs.append(df)
            break

# realign column names
for df in test_result_dfs:
    df.columns = [col.replace('At or Near', 'Near')
                                 .replace('Total Tested at Subgroup Level', 'Total Tested with Scores') 
                              for col in df.columns]
combined_df = pd.concat(test_result_dfs, axis=0, ignore_index=True, sort=False)
combined_df.shape

data/sb_ca2018.zip
data/sb_ca2017.zip
data/sb_ca2016.zip
data/sb_ca2015.zip


(408291, 32)

In [62]:
## entity is defined by CDS_CODE, which is composed of county code, district code, and school code
## each row has a unique CDS_CODE + Grade + Test Id
test_id_lkup_list = ['', 'SB - English Language Arts/Literacy', 'SB - Mathematics']
combined_df["CDS_CODE"] = (combined_df["County Code"].astype(str).str.pad(2, 'left', '0')
                  + combined_df["District Code"].astype(str).str.pad(5, 'left', '0') 
                  + combined_df["School Code"].astype(str).str.pad(7, 'left', '0')
                 )
columns_to_drop = ['Filler', 'Subgroup ID','Test Type'] + [col for col in combined_df.columns if col.startswith('Area ')]
combined_df["Test Name"] = df["Test Id"].apply(lambda x:test_id_lkup_list[x])
combined_df.drop(columns_to_drop, axis=1, inplace=True)
combined_df.head().T


,0,1,2,3,4
County Code,0,0,0,0,0
District Code,0,0,0,0,0
School Code,0,0,0,0,0
Test Year,2018,2018,2018,2018,2018
Total Tested At Entity Level,3180571,3187408,3187408,3180571,3180571
Total Tested with Scores,3177420,3184720,3184720,3177420,3177420
Grade,3,3,4,4,5
Test Id,1,2,2,1,1
CAASPP Reported Enrollment,445017,445018,463838,463838,469247
Students Tested,434468,436464,455619,453771,459434


## Read the entities file

In [69]:
zip_fn = 'data/ca_entities.zip'
encoding = "ISO-8859-1"
df = pd.read_csv(zip_fn, encoding=encoding)
df.head()

,County Code,District Code,School Code,Filler,Test Year,Type Id,County Name,District Name,School Name,Zip Code
0,0,0,0,NaN,2018,4,State of California,NaN,NaN,
1,1,0,0,NaN,2018,5,Alameda,NaN,NaN,
2,1,10017,0,NaN,2018,6,Alameda,Alameda County Office Of Education,NaN,
3,1,10017,112607,NaN,2018,9,Alameda,Envision Academy For Arts & Technology,Envision Academy For Arts & Technology,94612
4,1,10017,123968,NaN,2018,9,Alameda,Community School For Creative Education,Community School For Creative Education,94606


Type ID: 
* 4: Data is aggregated at the state level
* 5: Data is aggregated at the county level
* 6: Data is aggregated at the school district level
* 7: Data is aggregated at the public school level
* 9: Public charter school that receives funds directly from the state
* 10: Public charter school that receives funds from the school district

In [71]:
df["CDS_CODE"] = (df["County Code"].astype(str).str.pad(2, 'left', '0')
                  + df["District Code"].astype(str).str.pad(5, 'left', '0') 
                  + df["School Code"].astype(str).str.pad(7, 'left', '0')
                 )
type_id_dict = {4:'State', 5: 'County', 6: 'School District', 7:'Public School', 9:'State Funded Charter', 10:'District Funded Charter'}
df["Entity Type"] = df["Type Id"].apply(lambda x:type_id_dict[x])
df["School Name"] = df["School Name"].fillna("N/A")
df["District Name"] = df["District Name"].fillna("N/A")
df["County Name"] = df["County Name"].fillna("N/A")
df.head()

,County Code,District Code,School Code,Filler,Test Year,Type Id,County Name,District Name,School Name,Zip Code,CDS_CODE,Entity Type
0,0,0,0,NaN,2018,4,State of California,N/A,N/A,,00000000000000,State
1,1,0,0,NaN,2018,5,Alameda,N/A,N/A,,01000000000000,County
2,1,10017,0,NaN,2018,6,Alameda,Alameda County Office Of Education,N/A,,01100170000000,School District
3,1,10017,112607,NaN,2018,9,Alameda,Envision Academy For Arts & Technology,Envision Academy For Arts & Technology,94612,01100170112607,State Funded Charter
4,1,10017,123968,NaN,2018,9,Alameda,Community School For Creative Education,Community School For Creative Education,94606,01100170123968,State Funded Charter


In [72]:
# output the cleaned up entity file
df.drop(['Filler', 'Type Id'], axis=1, inplace=True)
df.to_csv(input_dir + "/" +'ca_entities_clean.csv')

## Split up the test result df into multiple files by entity type

In [95]:
df['Test Year'].value_counts()

2018    11333
Name: Test Year, dtype: int64

In [97]:
print("Shape before merge: ", combined_df.shape)
combined_df2 = pd.merge(combined_df, df.drop(['Test Year', 'County Code', 'District Code', 'School Code'], axis=1), how='inner', on=['CDS_CODE'])
print("Shape after merge: ", combined_df2.shape)

Shape before merge:  (408291, 19)
Shape after merge:  (392991, 24)


In [102]:
print("Records retained after merge: {:.2%}".format(combined_df2.shape[0]/combined_df.shape[0]))

Records retained after merge: 96.25%


In [103]:
combined_df2.to_csv(input_dir + "/" +'ca_smarter_balance_results_combined.csv')